In [42]:
#imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from path import Path

In [43]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [44]:
file_path = Path("../Resources/HR-Employee-Attrition.csv")
attrition_df = pd.read_csv(file_path)
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [45]:
#generate categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [46]:
#check number of unique values
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [47]:
#create onehotencoder instance
enc = OneHotEncoder(sparse=False)

#fit and transform
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

#add the encoded variable names 
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [48]:
#merge encoded features and drop originals
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat, 1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [49]:
#split preprocessed data into features and targets
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"],1).values

#split preprocessed data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [50]:
#create scaler instance
scaler = StandardScaler()

#fit scaler
X_scaler = scaler.fit(X_train)

#scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
#define the model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

#first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
#second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu"))

#output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#check summary of model
nn.summary()

Model: &quot;sequential_5&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 8)                 448       
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [52]:
#compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:
#train the model
fit_model = nn.fit(X_train, y_train, epochs=100)


Train on 1102 samples
Epoch 1/100
1102/1102 [==============================] - 2s 2ms/sample - loss: 270.2114 - accuracy: 0.7314
Epoch 2/100
1102/1102 [==============================] - 0s 127us/sample - loss: 120.6930 - accuracy: 0.6688
Epoch 3/100
1102/1102 [==============================] - 0s 131us/sample - loss: 78.5721 - accuracy: 0.6815
Epoch 4/100
1102/1102 [==============================] - 0s 131us/sample - loss: 59.1491 - accuracy: 0.6887
Epoch 5/100
1102/1102 [==============================] - 0s 134us/sample - loss: 47.3018 - accuracy: 0.7223
Epoch 6/100
1102/1102 [==============================] - 0s 138us/sample - loss: 38.6510 - accuracy: 0.7223
Epoch 7/100
1102/1102 [==============================] - 0s 151us/sample - loss: 30.4520 - accuracy: 0.7087
Epoch 8/100
1102/1102 [==============================] - 0s 141us/sample - loss: 25.3576 - accuracy: 0.7296
Epoch 9/100
1102/1102 [==============================] - 0s 144us/sample - loss: 18.7525 - accuracy: 0.7250
Epoch 

In [54]:
#evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 2.3590 - accuracy: 0.8614
Loss: 2.62156503615172, Accuracy: 0.8614130616188049


In [55]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 26/100
 384/1102 [=========&gt;....................] - ETA: 0s - loss: 0.2734 - accuracy: 0.8880
Epoch 00026: saving model to checkpoints/weights.26.hdf5
1102/1102 [==============================] - 0s 151us/sample - loss: 0.2927 - accuracy: 0.8848
Epoch 27/100
 448/1102 [===========&gt;..................] - ETA: 0s - loss: 0.2919 - accuracy: 0.8884
Epoch 00027: saving model to checkpoints/weights.27.hdf5
1102/1102 [==============================] - 0s 173us/sample - loss: 0.2907 - accuracy: 0.8866
Epoch 28/100
 288/1102 [======&gt;.......................] - ETA: 0s - loss: 0.2977 - accuracy: 0.8854
Epoch 00028: saving model to checkpoints/weights.28.hdf5
1102/1102 [==============================] - 0s 231us/sample - loss: 0.2880 - accuracy: 0.8884
Epoch 29/100
 320/1102 [=======&gt;......................] - ETA: 0s - loss: 0.2694 - accuracy: 0.8969
Epoch 00029: saving model to checkpoints/weights.29.hdf5
1102/1102 [==============================] - 0s 178us/sample - loss: 0.2858

In [56]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.1886 - accuracy: 0.8696
Loss: 0.3476372566074133, Accuracy: 0.8695651888847351


In [57]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [58]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.1886 - accuracy: 0.8696
Loss: 0.3476372566074133, Accuracy: 0.8695651888847351
